In [56]:
import pandas as pd
from glob import glob
from simpledbf import Dbf5
import os
import pyproj
import numpy as np

In [28]:
files = glob('C:\\Users\\jlandman\\Desktop\\newData\\AT-IT-KayHelfricht\\Fischer-etal-2015\\*ferner*\\*.dbf')

In [31]:
pop_list = ['Gliederferner', 'Langenferner', 'Rieserferner', 'Uebeltalferner', 'Weissbrunnferner', 'Weissbrunnferner1995']

In [74]:
for i in files:
    for j in pop_list:
        if j in i:
            files.remove(i)
files

['C:\\Users\\jlandman\\Desktop\\newData\\AT-IT-KayHelfricht\\Fischer-etal-2015\\Feuersteinferner\\Feuersteinferner.dbf',
 'C:\\Users\\jlandman\\Desktop\\newData\\AT-IT-KayHelfricht\\Fischer-etal-2015\\Grafferner\\Grafferner.dbf',
 'C:\\Users\\jlandman\\Desktop\\newData\\AT-IT-KayHelfricht\\Fischer-etal-2015\\Hangendenferner\\Hangendenferner.dbf',
 'C:\\Users\\jlandman\\Desktop\\newData\\AT-IT-KayHelfricht\\Fischer-etal-2015\\Hohenferner\\ed_Hohenferner.dbf',
 'C:\\Users\\jlandman\\Desktop\\newData\\AT-IT-KayHelfricht\\Fischer-etal-2015\\Laaserferner\\Laaserferner.dbf',
 'C:\\Users\\jlandman\\Desktop\\newData\\AT-IT-KayHelfricht\\Fischer-etal-2015\\Langtaufererferner\\Langtaufererferner.dbf',
 'C:\\Users\\jlandman\\Desktop\\newData\\AT-IT-KayHelfricht\\Fischer-etal-2015\\Matscherferner\\Matscherferner.dbf',
 'C:\\Users\\jlandman\\Desktop\\newData\\AT-IT-KayHelfricht\\Fischer-etal-2015\\Oberer_Ortlerferner\\obererOrtlerferner.dbf',
 'C:\\Users\\jlandman\\Desktop\\newData\\AT-IT-KayHelfri

In [83]:
thin_dict = {'Feuersteinferner': 25,
             'Grafferner': 15,
             'Hangendenferner': 25,
             'ed_Hohenferner':25,
             'Laaserferner':65,
             'Langtaufererferner':20,
             'Matscherferner':20,
             'obererOrtlerferner':2,
             'Schranferner':10,
             'Suldenferner':20
    
}

date_dict = {'Feuersteinferner': 20130608,
             'Grafferner': 20130612,
             'Hangendenferner': 20130608,
             'ed_Hohenferner':20130514,
             'Laaserferner':20130513,
             'Langtaufererferner':20130613,
             'Matscherferner':20130612,
             'obererOrtlerferner':np.nan,
             'Schranferner':20130514,
             'Suldenferner':20130611}


In [80]:
# Some constants
gtd_id = np.nan # GlaThiDa_ID
punit = 'IT' # political unit
gname = ''
pid = np.nan
thickness_unc = np.nan
data_flag = np.nan
remarks = ''

In [81]:
utm32 = pyproj.Proj(init='epsg:32632') #WGS84 UTM32N
latlon = pyproj.Proj(init='epsg:4326')  #WGS84 lat/lon

In [94]:
for file in files:
    fname = os.path.basename(file)
    dbf = Dbf5(file)
    data = dbf.to_dataframe()
    data.columns = map(str.lower, data.columns) # make all columns lowercase, that's easier
    data = data.groupby(data.index // thin_dict[fname.split('.')[0]]).mean() # average over every x values as specified above
    
    try:
        # project UTM values to lat/lon
        xs = data.x.values
        ys = data.y.values
        
        x1,y1 = p1(xs, ys)
        lons, lats = pyproj.transform(utm32,latlon,xs,ys)
        
        data['POINT_LAT'] = lats
        data['POINT_LON'] = lons
        
        data['GlaThiDa_ID'] = gtd_id
        data['POLITICAL_UNIT'] = punit
        data['GLACIER_NAME'] = fname.split('.')[0].upper()
        data['SURVEY_DATE'] = date_dict[fname.split('.')[0]]
        data['POINT_ID'] = range(1,len(data)+1)
        
        try:
            data['ELEVATION'] = (data['ed'] + data['ug']).map('{:.0f}'.format)
        except KeyError:
            data['ELEVATION'] = np.nan
            
        data['THICKNESS'] = data['ed'].map('{:.0f}'.format)
        data['THICKNESS_UNCERTAINTY'] = thickness_unc
        data['DATA_FLAG'] = data_flag
        data['REMARKS'] = remarks + ' Every %s points from the original series have been averaged.' % thin_dict[fname.split('.')[0]]
        
        data = data[['GlaThiDa_ID','POLITICAL_UNIT','GLACIER_NAME','SURVEY_DATE','POINT_ID','POINT_LAT','POINT_LON',
                     'ELEVATION','THICKNESS','THICKNESS_UNCERTAINTY','DATA_FLAG','REMARKS']]
        data.to_excel(fname.split('.')[0]+'.xlsx', index=False)
    except AttributeError:
        print('Table %s has no coordinates' % fname)

Table obererOrtlerferner.dbf has no coordinates
